# 1. Import dependencies

In [2]:
#Modules en lien avec Gym
import gymnasium as gym

#Modules utiles
import sys
sys.path.append('/Users/mathiasperez/Documents/GitHub/PSC_Stratolia/Algo Naif')
from parcours import parcours_a_Z, distance_destination, case, ventU_ventV

import os
import numpy as np
import random
import os

#Modules en lien avec Stable_baseline3
#IMPORT THE TYPE OF AGENT U WANT TO USE
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Create the environnement

In [ ]:
class Balloon(gym.Env):
    def __init__(self):
        super(Balloon, self).__init__()

        goal_long = 2.65
        goal_lat = 34.54

        temps_exploration = 3600
        
        # Define the action space
        self.action_space = gym.spaces.Discrete(3)  
        """
        Action 0 : You do nothing and let the balloon follow the wind for temps_exploration seconds
        Action 1 : You push the balloon upwards (goes up in altitude) (instanneous action)
        Action 2 : You push the balloon downwards (goes down in altitude) (instanneous action)
        """
        
        # Define the observation space
        self.observation_space = gym.spaces.Tuple(( 
            gym.spaces.Box(low=0, high=86400, shape=(1,), dtype=np.float32), # time (in special format)
            gym.spaces.Discrete(18), #Altitude (17 different positions : no need for more precisions)
            gym.spaces.Box(low=0, high=180, shape=(1,), dtype=np.float32),  #Longitude
            gym.spaces.Box(low=0, high=90, shape=(1,), dtype=np.float32)   #Latitude
            ))
        
    def reset(self):
        # Reset the environment to its initial state
        # Resetting the ballon to its initial position : 73 boulevard des maréchaux
        self.state = (np.array([0], dtype=np.float32), 
                      np.array([0], dtype=np.float32), 
                      0, 
                      #special time 
                      )
        return self.state, {}
    
    def get_terminated(self, state):
        
        pass
    
    def get_reward(self, state):
        pass

    
    def step(self, action):
        # Take a step in the environment based on the given action
        if action == 0:
            #You decide to follow the wind flow :
            

            self.state[0] += 1
        elif action == 1:
            #You go up 1 case in altitude
            self.state[1] += 1
        else :
            #You go down one case in altitude
            pass
    

        # Calculate the reward
        reward = get_reward(state)
        
        # Check if the episode has ended
        done = get_terminated(state)
        
        # Check if the episode has been truncated
        truncated = False
        
        # Return the observation, reward, done flag, truncated flag, and info
        return self.state, reward, done, truncated, {}
    
    def render(self, mode='human'):
        # Render the environment
        print("State:", self.state)


    
